In [40]:
import numpy as np
np.set_printoptions(precision=3)
sigma = lambda x: 1/(1+np.exp(-x))
d_sigma= lambda x: sigma(x)*(1-sigma(x))
eta=0.5
def F_prop(X, W):
    L=len(W)
    X=X=np.append(X, np.ones((X.shape[0],1)), axis = 1)
    F_X = {'X0':X.T}
    for h in range(1, L+1):
        F_X[f'Y{h}']=np.dot(W[f'W{h}'], F_X[f'X{h-1}'])
        if h!=L:
            F_X[f'X{h}']=np.concatenate((sigma(F_X[f'Y{h}']), np.ones((1,X.shape[0]))), axis = 0)
        else:
            F_X[f'X{h}']=sigma(F_X[f'Y{h}'])
    return F_X

def F_star_W(X,y, W):
    L=len(W)
    F_X = F_prop(X, W)
    FX_star={'X*2':(F_X['X2']-y)}
    Y_star={}
    ##
    FX_star['Y*2']=FX_star['X*2']*(d_sigma(F_X['Y2']))    
    W['W2']= W['W2'] -eta*FX_star['Y*2'].dot(F_X['X1'].T) 
    ##
    FX_star['X*1']=np.delete(W['W2'], -1, axis = 1).T.dot(FX_star['Y*2'])
    FX_star['Y*1']=FX_star['X*1']*(d_sigma(F_X['Y1']))
    W['W1']= W['W1'] -eta*FX_star['Y*1'].dot(F_X['X0'].T)     
    return W
def F_star(X,y, W):
    L=len(W)
    F_X = F_prop(X, W)
    FX_star={'X*2':(F_X['X2']-y)}      
    FX_star['Y*2']=FX_star['X*2']*(d_sigma(F_X['Y2']))
    ##
    W['W2']= W['W2'] -eta*FX_star['Y*2'].dot(F_X['X1'].T) 
    #
    FX_star['X*1']=np.delete(W['W2'], -1, axis = 1).T.dot(FX_star['Y*2'])
    FX_star['Y*1']=FX_star['X*1']*(d_sigma(F_X['Y1']))        
    return FX_star

def F_star_prop(X,y, W):
    L=len(W)
    F_X = F_prop(X, W)
    FX_star={f'X*{L}':(F_X[f'X{L}']-y)}
    Y_star={}
    for h in reversed(range(1, L+1)):
        FX_star[f'Y*{h}']=FX_star[f'X*{h}']*(d_sigma(F_X[f'Y{h}']))
        FX_star[f'X*{h-1}']=np.delete(W[f'W{h}'], -1, axis = 1).T.dot(FX_star[f'Y*{h}'])
    return FX_star
def Grad_star(X,y, W):
    L=len(W)
    F_X = F_prop(X, W)
    FX_star=F_star_prop(X,y, W)
    Grad={}
    for h in range(1, L+1):
        Grad[f'W{h}']=W[f'W{h}']-eta*FX_star[f'Y*{h}'].dot(F_X[f'X{h-1}'].T)
    return Grad

In [20]:
# Learning the XOR dataset
X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
y = np.array([0, 1, 1, 0]) 
#X = np.array([[0, 0]]); y=np.array([[1]])
W1=np.array([[1, -1, 1], [-1, 1, -1]])
W2=np.array([[1, -1, 1]])
W={'W1':W1, 'W2':W2}

In [21]:
F_prop(X, W)

{'X0': array([[0., 0., 1., 1.],
        [0., 1., 0., 1.],
        [1., 1., 1., 1.]]),
 'Y1': array([[ 1.,  0.,  2.,  1.],
        [-1.,  0., -2., -1.]]),
 'X1': array([[0.731, 0.5  , 0.881, 0.731],
        [0.269, 0.5  , 0.119, 0.269],
        [1.   , 1.   , 1.   , 1.   ]]),
 'Y2': array([[1.462, 1.   , 1.762, 1.462]]),
 'X2': array([[0.812, 0.731, 0.853, 0.812]])}

In [22]:
FX_star=F_star_prop(X,y, W)

In [23]:
W

{'W1': array([[ 1, -1,  1],
        [-1,  1, -1]]),
 'W2': array([[ 1, -1,  1]])}

In [26]:
X=FX_star['X*0'].T

In [27]:
F_prop(X, W)

{'X0': array([[ 0.049, -0.026, -0.004,  0.049],
        [-0.049,  0.026,  0.004, -0.049],
        [ 1.   ,  1.   ,  1.   ,  1.   ]]),
 'Y1': array([[ 1.098,  0.947,  0.992,  1.098],
        [-1.098, -0.947, -0.992, -1.098]]),
 'X1': array([[0.75 , 0.721, 0.73 , 0.75 ],
        [0.25 , 0.279, 0.27 , 0.25 ],
        [1.   , 1.   , 1.   , 1.   ]]),
 'Y2': array([[1.5  , 1.441, 1.459, 1.5  ]]),
 'X2': array([[0.818, 0.809, 0.811, 0.818]])}

In [29]:
W=Grad_star(X,y, W)
W

{'W1': array([[ 0.999, -0.999,  0.983],
        [-0.999,  0.999, -0.983]]),
 'W2': array([[ 0.93 , -1.022,  0.907]])}

In [37]:
W=Grad_star(X,y, W)
FX_star=F_star_prop(X,y, W)
X=X-FX_star['X*0'].T
F_prop(X, W)

{'X0': array([[-0.418,  0.185,  0.201, -0.418],
        [ 0.418, -0.185, -0.201,  0.418],
        [ 1.   ,  1.   ,  1.   ,  1.   ]]),
 'Y1': array([[ 0.014,  1.254,  1.287,  0.014],
        [ 0.069, -1.204, -1.237,  0.069]]),
 'X1': array([[0.503, 0.778, 0.784, 0.503],
        [0.517, 0.231, 0.225, 0.517],
        [1.   , 1.   , 1.   , 1.   ]]),
 'Y2': array([[-0.065,  0.464,  0.475, -0.065]]),
 'X2': array([[0.484, 0.614, 0.617, 0.484]])}

In [38]:
W

{'W1': array([[ 1.028, -1.028,  0.873],
        [-1.055,  1.055, -0.813]]),
 'W2': array([[ 0.587, -1.286,  0.304]])}

In [39]:
X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
F_prop(X, W)

{'X0': array([[0., 0., 1., 1.],
        [0., 1., 0., 1.],
        [1., 1., 1., 1.]]),
 'Y1': array([[ 0.873, -0.155,  1.901,  0.873],
        [-0.813,  0.242, -1.868, -0.813]]),
 'X1': array([[0.705, 0.461, 0.87 , 0.705],
        [0.307, 0.56 , 0.134, 0.307],
        [1.   , 1.   , 1.   , 1.   ]]),
 'Y2': array([[ 0.323, -0.145,  0.643,  0.323]]),
 'X2': array([[0.58 , 0.464, 0.655, 0.58 ]])}

In [41]:
# Learning the XOR dataset
X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
y = np.array([0, 1, 1, 0]) 
#X = np.array([[0, 0]]); y=np.array([[1]])
W1=np.array([[1, -1, 1], [-1, 1, -1]])
W2=np.array([[1, -1, 1]])
W={'W1':W1, 'W2':W2}
##
X2=F_prop(X, W)['X2']
##
X2_c=np.where(X2>=0.5,1,0)
k=0
while (X2_c -y).any() != 0 :
#for k in range(100):
    print('X',X)
    X2=F_prop(X, W)['X2']
    X2_c=np.where(X2>=0.5,1,0)
    FX_star=F_star_prop(X,y, W)
    
    
    print('X2',X2,k+1)
    W=F_star_W(X,y, W)
    #W=Grad_star(X,y, W)
    X=X-FX_star['X*0'].T
    k+=1
    print("================================")

X [[0 0]
 [0 1]
 [1 0]
 [1 1]]
X2 [[0.812 0.731 0.853 0.812]] 1
X [[-0.049  0.049]
 [ 0.026  0.974]
 [ 1.004 -0.004]
 [ 0.951  1.049]]
X2 [[0.781 0.707 0.833 0.78 ]] 2
X [[-0.102  0.103]
 [ 0.056  0.944]
 [ 1.009 -0.009]
 [ 0.898  1.103]]
X2 [[0.746 0.681 0.81  0.743]] 3
X [[-0.159  0.162]
 [ 0.088  0.911]
 [ 1.015 -0.015]
 [ 0.839  1.163]]
X2 [[0.706 0.654 0.784 0.7  ]] 4
X [[-0.219  0.226]
 [ 0.123  0.873]
 [ 1.022 -0.022]
 [ 0.778  1.228]]
X2 [[0.662 0.627 0.758 0.654]] 5
X [[-0.281  0.292]
 [ 0.161  0.832]
 [ 1.03  -0.032]
 [ 0.715  1.296]]
X2 [[0.618 0.602 0.733 0.607]] 6
X [[-0.343  0.36 ]
 [ 0.202  0.787]
 [ 1.041 -0.043]
 [ 0.653  1.365]]
X2 [[0.575 0.582 0.712 0.561]] 7
X [[-0.403  0.429]
 [ 0.246  0.737]
 [ 1.052 -0.056]
 [ 0.593  1.432]]
X2 [[0.535 0.567 0.695 0.518]] 8
X [[-0.462  0.496]
 [ 0.292  0.685]
 [ 1.065 -0.07 ]
 [ 0.537  1.497]]
X2 [[0.499 0.558 0.684 0.48 ]] 9


In [42]:
W

{'W1': array([[ 0.966, -1.093,  0.876],
        [-0.962,  1.158, -0.808]]),
 'W2': array([[ 0.636, -1.297,  0.346]])}

In [44]:
F_prop(X, W)

{'X0': array([[-0.518,  0.339,  1.078,  0.484],
        [ 0.561,  0.63 , -0.086,  1.559],
        [ 1.   ,  1.   ,  1.   ,  1.   ]]),
 'Y1': array([[-0.238,  0.515,  2.012, -0.361],
        [ 0.34 , -0.405, -1.944,  0.532]]),
 'X1': array([[0.441, 0.626, 0.882, 0.411],
        [0.584, 0.4  , 0.125, 0.63 ],
        [1.   , 1.   , 1.   , 1.   ]]),
 'Y2': array([[-0.132,  0.225,  0.745, -0.21 ]]),
 'X2': array([[0.467, 0.556, 0.678, 0.448]])}

In [85]:
W1=np.array([[1, -1, 1], [-1, 1, 0]])
W1@W1.T

array([[ 3, -2],
       [-2,  2]])

In [75]:
W2=np.array([[1, -1, 1]])
W2@W2.T

array([[3]])

In [126]:
((np.sqrt(17)+5)/2)**2

20.807764064044157

In [147]:
# Learning the XOR dataset
X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
y = np.array([0, 1, 1, 0]) 
#X = np.array([[0, 0]]); y=np.array([[1]])*np.sqrt(6)
W1=np.array([[1, -1, 1], [-1, 1, 0]])*(20)
W2=np.array([[1, -1, 1]])
W={'W1':W1, 'W2':W2}
##
X2=F_prop(X, W)['X2']
##
X2_c=np.where(X2>=0.5,1,0)
k=0
while (X2_c -y).any() != 0 :
#for k in range(10):
    #print('X',X)
    X2=F_prop(X, W)['X2']
    X2_c=np.where(X2>=0.5,1,0)
    FX_star=F_star_prop(X,y, W)
    
    
    print('X2=',X2,k+1)
    #W=F_star_W(X,y, W)
    W=Grad_star(X,y, W)
    X=X-FX_star['X*0'].T##(X0-Xstar0) as input
    k+=1
    print("================================")

X2= [[0.818 0.622 0.881 0.818]] 1
X2= [[0.481 0.713 0.862 0.481]] 2


In [148]:
# Learning the XOR dataset
X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
y = np.array([0, 1, 1, 0])          
# Neural network with the architecture A[3,2,1]
# W1=np.array([[1, 0, 0], [0, 1, 0]])
# W2=np.array([[0, 0, 1]])

W1=np.array([[1, -1, 1], [-1, 1, 0]])*(20)
W2=np.array([[1, -1, 1]])
W={'W1':W1, 'W2':W2}
W={'W1':W1, 'W2':W2}
##
X2=F_prop(X, W)['X2']
X2_c=np.where(X2>=0.5,1,0)
k=0
while (X2_c -y).any() != 0 :
   # print("================================")
    X2=F_prop(X, W)['X2']
    X2_c=np.where(X2>=0.5,1,0)
    #FX_star=F_star_prop(X,y, W)
    
    
    print('X2=',X2,k+1)
    #W=F_star_W(X,y, W)
    W=Grad_star(X,y, W)
    #X=X-FX_star['X*0'].T##(X0-Xstar0) as input
    k+=1
    print("================================")

X2= [[0.818 0.622 0.881 0.818]] 1
X2= [[0.789 0.592 0.862 0.788]] 2
X2= [[0.756 0.558 0.841 0.753]] 3
X2= [[0.718 0.523 0.816 0.713]] 4
X2= [[0.677 0.488 0.788 0.669]] 5
X2= [[0.636 0.455 0.76  0.625]] 6
X2= [[0.597 0.426 0.732 0.583]] 7
X2= [[0.562 0.402 0.706 0.545]] 8
X2= [[0.533 0.383 0.685 0.513]] 9
X2= [[0.509 0.369 0.667 0.488]] 10
X2= [[0.491 0.359 0.653 0.468]] 11
X2= [[0.477 0.352 0.643 0.452]] 12
X2= [[0.466 0.348 0.635 0.44 ]] 13
X2= [[0.458 0.345 0.629 0.431]] 14
X2= [[0.452 0.344 0.624 0.424]] 15
X2= [[0.447 0.344 0.621 0.418]] 16
X2= [[0.444 0.345 0.619 0.414]] 17
X2= [[0.441 0.346 0.618 0.41 ]] 18
X2= [[0.439 0.348 0.617 0.408]] 19
X2= [[0.438 0.35  0.617 0.405]] 20
X2= [[0.436 0.352 0.617 0.404]] 21
X2= [[0.435 0.354 0.617 0.402]] 22
X2= [[0.434 0.356 0.617 0.4  ]] 23
X2= [[0.433 0.359 0.617 0.399]] 24
X2= [[0.433 0.361 0.618 0.398]] 25
X2= [[0.432 0.363 0.618 0.397]] 26
X2= [[0.431 0.365 0.619 0.396]] 27
X2= [[0.43  0.367 0.62  0.395]] 28
X2= [[0.43  0.37  0.62  0.394

In [118]:
X2=F_prop(X, W)['X2']
##
X2_c=np.where(X2>=0.5,1,0)
X2_c

array([[0, 1, 1, 0]])

In [119]:
W

{'W1': array([[ 13.999, -14.005,  13.994],
        [-13.985,  14.015,   0.03 ]]),
 'W2': array([[ 0.866, -1.194,  0.771]])}

In [7]:
X = np.array([[0, 0]]); y=np.array([[1]])
W1=np.array([[1, 0, 0], [0, 1, 0]])
W2=np.array([[0, 0, 1]])
W={'W1':W1, 'W2':W2}
##
##
X2=F_prop(X, W)['X2']
X2_c=np.where(X2>=0.5,1,0)
k=0
while (X2_c -y).any() != 0 :
   # print("================================")
    #print(X, X2)
    #print("================================================================================================")
    
    
    FX_star=F_star_prop(X,y, W)
    X=FX_star['X*0'].T
    
    W=F_star_W(X,y, W)
    
    X2=F_prop(X, W)['X2']
    X2_c=np.where(X2>=0.5,1,0)
    print(X, X2)
    print("================================================================================================")
    k+=1


In [52]:
y = np.array([0])
FX_star=F_star_prop(X,y, W)
FX_star

{'X*2': array([[0.731]]),
 'Y*2': array([[0.144]]),
 'X*1': array([[0.],
        [0.]]),
 'Y*1': array([[0.],
        [0.]]),
 'X*0': array([[0.],
        [0.]])}

In [53]:
W=F_star_W(X,y, W)

In [54]:
X=FX_star['X*0']

In [56]:
F_prop(X.T, W)

{'X0': array([[0.],
        [0.],
        [1.]]),
 'Y1': array([[0.048],
        [0.048]]),
 'X1': array([[0.512],
        [0.512],
        [1.   ]]),
 'Y2': array([[0.066]]),
 'X2': array([[0.516]])}

In [123]:
# Learning the XOR dataset
X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
y = np.array([0, 1, 1, 0])          
# Neural network with the architecture A[3,2,1]
# W1=np.array([[1, 0, 0], [0, 1, 0]])
# W2=np.array([[0, 0, 1]])

W1=np.array([[1, -1, 1], [-1, 1, 0]])*(14)
W2=np.array([[1, -1, 1]])
W={'W1':W1, 'W2':W2}
W={'W1':W1, 'W2':W2}
##
X2=F_prop(X, W)['X2']
X2_c=np.where(X2>=0.5,1,0)
k=0
while (X2_c -y).any() != 0 :
   # print("================================")
    print( X2,k+1)
    print("================================================================================================")
    W=F_star_W(X,y, W)
    X2=F_prop(X, W)['X2']
    X2_c=np.where(X2>=0.5,1,0)
    k+=1

[[0.818 0.622 0.881 0.818]] 1
[[0.789 0.591 0.862 0.788]] 2
[[0.756 0.558 0.841 0.752]] 3
[[0.718 0.523 0.816 0.712]] 4
[[0.677 0.488 0.788 0.669]] 5
[[0.635 0.455 0.759 0.624]] 6
[[0.596 0.425 0.732 0.582]] 7
[[0.561 0.401 0.706 0.544]] 8
[[0.532 0.382 0.685 0.513]] 9
[[0.509 0.368 0.667 0.487]] 10
[[0.49  0.359 0.653 0.467]] 11
[[0.476 0.352 0.642 0.451]] 12
[[0.466 0.348 0.634 0.439]] 13
[[0.458 0.345 0.628 0.43 ]] 14
[[0.451 0.344 0.624 0.423]] 15
[[0.447 0.344 0.621 0.417]] 16
[[0.443 0.345 0.619 0.413]] 17
[[0.441 0.346 0.618 0.41 ]] 18
[[0.439 0.348 0.617 0.407]] 19
[[0.437 0.35  0.616 0.405]] 20
[[0.436 0.352 0.616 0.403]] 21
[[0.434 0.354 0.616 0.401]] 22
[[0.433 0.357 0.617 0.4  ]] 23
[[0.433 0.359 0.617 0.398]] 24
[[0.432 0.361 0.617 0.397]] 25
[[0.431 0.364 0.618 0.396]] 26
[[0.43  0.366 0.618 0.395]] 27
[[0.43  0.368 0.619 0.394]] 28
[[0.429 0.37  0.62  0.393]] 29
[[0.428 0.373 0.62  0.392]] 30
[[0.427 0.375 0.621 0.391]] 31
[[0.427 0.377 0.622 0.39 ]] 32
[[0.426 0.379 0.6

In [43]:
# Learning the XOR dataset
X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
y = np.array([0, 1, 1, 0])
# Neural network with the architecture A[3,2,1]
import math
np.random.seed(0)
scale = 1/max(1., (2+2)/2.)
limit = math.sqrt(3.0 * scale)
W1 = np.random.uniform(-limit, limit, size=(2,3))
W2=np.random.uniform(-limit, limit, size=(1,3))

####W1=np.array([[1, 0, 0], [0, 1, 0]])
##W2=np.array([[0, 0, 1]])
W={'W1':W1, 'W2':W2}
##
X2=F_prop(X, W)['X2']
X2_c=np.where(X2>=0.5,1,0)
k=0
while (X2_c -y).any() != 0 :
   # print("================================")
    print(W, X2, X2_c,k)
    print("================================================================================================")
    #W=Grad_star(X,y, W)
    W=F_star_W(X,y, W)
    X2=F_prop(X, W)['X2']
    X2_c=np.where(X2>=0.5,1,0)
    k+=1

{'W1': array([[ 0.11956818,  0.52710415,  0.25171784],
       [ 0.1099409 , -0.18700679,  0.35736613]]), 'W2': array([[-0.15287949,  0.95964395,  1.13573718]])} [[0.83401921 0.82513663 0.83688145 0.82831476]] [[1 1 1 1]] 0
{'W1': array([[ 0.12156539,  0.52909491,  0.25614968],
       [ 0.09925049, -0.19724955,  0.33684796]]), 'W2': array([[-0.2119747 ,  0.9058162 ,  1.04286188]])} [[0.8102829  0.79927113 0.81263464 0.80196212]] [[1 1 1 1]] 1
{'W1': array([[ 0.1241875 ,  0.53171107,  0.26200698],
       [ 0.08878653, -0.20719919,  0.31691042]]), 'W2': array([[-0.27292374,  0.85112914,  0.94726458]])} [[0.78337878 0.76998157 0.78507664 0.77204461]] [[1 1 1 1]] 2
{'W1': array([[ 0.12735815,  0.53488191,  0.26917735],
       [ 0.07885117, -0.21655634,  0.29810505]]), 'W2': array([[-0.33421342,  0.7969669 ,  0.85136097]])} [[0.75386643 0.73792174 0.75477838 0.73923228]] [[1 1 1 1]] 3
{'W1': array([[ 0.13089933,  0.53844027,  0.27735066],
       [ 0.06974595, -0.2250295 ,  0.28097074]]), 'W2

In [62]:
# Learning the XOR dataset
X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
y = np.array([0, 1, 1, 0]) 
#X = np.array([[0, 0]]); y=np.array([[1]])
#W1=np.array([[1, -1, 1], [-1, 1, -1]])
#W1=np.random.uniform(0, 1,(2, 3))
#W2=np.random.uniform(0, 1,(1, 3))
#W2=np.array([[1, -1, 1]])
#W1=np.random.randn(2,3)*np.sqrt(2/3)
#W2=np.random.randn(1,3)*np.sqrt(2/3)
import math
np.random.seed(0)
scale = 1/max(1., (2+2)/2.)
limit = math.sqrt(3.0 * scale)
W1 = np.random.uniform(-limit, limit, size=(2,3))
W2=np.random.uniform(-limit, limit, size=(1,3))



W={'W1':W1, 'W2':W2}
##np.random.uniform(0, 1, (x, y, z))
X2=F_prop(X, W)['X2']
##np.random.randn(layer_size[l],layer_size[l-1])*np.sqrt(2/layer_size[l-1])
X2_c=np.where(X2>=0.5,1,0)
k=0
while (X2_c -y).any() != 0 :
#for k in range(10):
    print('X',X)
    X2=F_prop(X, W)['X2']
    X2_c=np.where(X2>=0.5,1,0)
    FX_star=F_star_prop(X,y, W)
    
    
    print('X2',X2,k+1)
    #W=F_star_W(X,y, W)
    W=Grad_star(X,y, W)
    X=X-FX_star['X*0'].T
    k+=1
    print("================================")

X [[0 0]
 [0 1]
 [1 0]
 [1 1]]
X2 [[0.834 0.825 0.837 0.828]] 1
X [[-2.431e-03  7.307e-03]
 [ 5.612e-04  9.984e-01]
 [ 1.000e+00 -1.380e-03]
 [ 9.974e-01  1.007e+00]]
X2 [[0.81  0.799 0.813 0.802]] 2
X [[-0.004  0.016]
 [ 0.001  0.996]
 [ 1.001 -0.003]
 [ 0.995  1.016]]
X2 [[0.783 0.77  0.785 0.772]] 3
X [[-0.006  0.027]
 [ 0.002  0.993]
 [ 1.001 -0.006]
 [ 0.994  1.026]]
X2 [[0.753 0.738 0.755 0.739]] 4
X [[-0.006  0.039]
 [ 0.002  0.989]
 [ 1.001 -0.01 ]
 [ 0.993  1.037]]
X2 [[0.722 0.704 0.723 0.704]] 5
X [[-0.006  0.052]
 [ 0.002  0.984]
 [ 1.001 -0.015]
 [ 0.992  1.05 ]]
X2 [[0.691 0.671 0.691 0.669]] 6
X [[-0.006  0.067]
 [ 0.002  0.977]
 [ 1.001 -0.022]
 [ 0.993  1.063]]
X2 [[0.661 0.639 0.661 0.636]] 7
X [[-0.004  0.083]
 [ 0.001  0.969]
 [ 1.001 -0.03 ]
 [ 0.993  1.077]]
X2 [[0.633 0.611 0.634 0.606]] 8
X [[-2.849e-03  9.941e-02]
 [ 5.252e-04  9.591e-01]
 [ 9.996e-01 -3.908e-02]
 [ 9.944e-01  1.091e+00]]
X2 [[0.61  0.587 0.61  0.581]] 9
X [[-8.628e-04  1.163e-01]
 [-5.882e-04 